In [1]:
import mcnp_input_reader as mir

In [2]:
inputfile = 'input.i'

In [3]:
!cat $inputfile

Water phantom
c Cell cards
1 1 -1.0 2 -1 4 -3 5 -6 fill=1 imp:p,e 1
c Water phantom
10 1 -1.0 2 -1 4 -3 5 -10 lat=1 u=1 imp:p,e 1
c Lattice call for dose deposition
100 2 -0.001293 (101 -102 -100) #1 imp:p,e 1
c Air around
999 0 -101:102:100 imp:p=0 imp:e=0
c Void cell

c Surface cards
c Water phantom
1 px 5
2 px -5
3 py 5
4 py -5
5 pz 10
6 pz 20
c Lattice cell
10 pz 11
c Cylinder around the problem
100 cz 9
101 pz -0.1
102 pz 25

c Data cards
mode p e
c Materials
m1 1000. 2
 8000. 1 $Water
m2 7014.
 -0.755636 $air (US S. Atm at sea level)
8016.
 -0.231475 18000.
 -0.012889
c Source cards
sdef pos=0 0 0 x=d1 y=d2 z=0 erg=10 par=2 $
si1 -6 6
sp1 0 1
si2 -6 6
sp2 0 1
c Tallies
F1:p 6
e1 1 99i 10 $ Energy spectrum, step 0.1 MV
*F8:p (10<10[0:0 0:0 -9:0]) $Depth dose tally
cut:e j 0.1 $100 keV (default is 1 keV)
cut:p j 0.01 $10 keV (default is 1 keV)
PHYS:P 4j 1 $turns off Doppler broadening
nps 1000000


In [4]:
mcnp_input = mir.read_file(inputfile)

In [5]:
mcnp_input

Title: Water phantom

The input contains:
# of cells: 4
# of surfaces: 10
# of materials: 2
# of transformations: 0

In [6]:
mcnp_input.cells

<MCNPCells>: [1, 10, 100, 999]

In [7]:
mcnp_input.cells[10]

MCNPCell(id=10, mat_id=1, density=-1.0, geometry='2 -1 4 -3 5 -10', fill_id=0, fill_type='', universe_id=1, imp_p=1.0, imp_n=0.0, imp_e=1.0, transf='', transf_id=0, lat=1, start_line=4, end_line=5, comment='Lattice call for dose deposition', input_cell_description='10 1 -1.0 2 -1 4 -3 5 -10 lat=1 u=1 imp:p,e 1\nc Lattice call for dose deposition', surfaces={1, 2, 3, 4, 5, 10}, not_cells=set())

In [8]:
print(mcnp_input.cells[10])

<MCNPCell>
id = 10
mat_id = 1
density = -1.0
geometry = 2 -1 4 -3 5 -10
surfaces = {1, 2, 3, 4, 5, 10}
not_cells = set()
fill_id = 0
fill_type = 
universe_id = 1
imp_p = 1.0
imp_n = 0.0
imp_e = 1.0
transf = 
transf_id = 0
lat = 1
start_line = 4
end_line = 5
comment = Lattice call for dose deposition
input_cell_description: 
10 1 -1.0 2 -1 4 -3 5 -10 lat=1 u=1 imp:p,e 1
c Lattice call for dose deposition



In [9]:
mcnp_input.surfaces

<MCNPSurfs>: [1, 2, 3, 4, 5, 6, 10, 100, 101, 102]

In [10]:
mcnp_input.surfaces[100]

MCNPSurf(id=100, surface_type='CZ', surface_parameters='9', transf_id=0, start_line=22, end_line=22, input_surface_description='100 CZ 9')

In [11]:
mcnp_input.materials

<MCNPMaterials>: [1, 2]

In [12]:
mcnp_input.materials[1]

MCNPMaterial(id=1, composition='1000. 2 8000. 1', start_line=29, end_line=30, input_material_description='M1 1000. 2\n 8000. 1 $WATER')

## Cell methods

In [13]:
mcnp_input.cells.get_surfaces()

{1, 2, 3, 4, 5, 6, 10, 100, 101, 102}

In [14]:
mcnp_input.cells.get_materials()

{1, 2}

In [15]:
mcnp_input.cells.get_fill_ids()

{1}

In [16]:
mcnp_input.cells.get_universe_ids()

{1}

In [17]:
mcnp_input.cells.get_transformations()

set()

## Filter Capabilities

In [18]:
mcnp_input.cells.filter(lambda cell: cell.mat_id == 1)

<MCNPCells>: [1, 10]

In [19]:
mcnp_input.cells.filter(lambda cell: cell.mat_id == 2)

<MCNPCells>: [100]

In [20]:
mcnp_input.cells.filter(lambda cell: cell.id < 20)

<MCNPCells>: [1, 10]

In [21]:
mcnp_input.cells.filter(lambda cell: cell.start_line < 7)

<MCNPCells>: [1, 10, 100]

In [22]:
mcnp_input.cells.filter(lambda cell: cell.start_line in range(7, 20))

<MCNPCells>: [999]

In [23]:
mcnp_input.cells.filter(lambda cell: cell.mat_id == 2).get_surfaces()

{100, 101, 102}

In [24]:
mcnp_input.surfaces.filter(lambda surf: surf.id < 10)

<MCNPSurfs>: [1, 2, 3, 4, 5, 6]

## Extract method

In [25]:
mcnp_input.extract(lambda cell: cell.mat_id == 2)

Title: Extracted from Water phantom

The input contains:
# of cells: 1
# of surfaces: 3
# of materials: 1
# of transformations: 0

In [26]:
mcnp_input.extract(lambda cell: cell.universe_id == 1).get_start_end_lines() # list of start-stop line numbers

[[4, 6], [13, 14], [14, 15], [15, 16], [16, 17], [17, 18], [20, 22], [29, 31]]

## to_csv method

In [27]:
mcnp_input.cells.to_csv('cells.csv', ['id','mat_id','density','fill_id'])

In [28]:
!cat cells.csv